In [1]:
!pip install beautifulsoup4
!pip install requests
!pip install openai --upgrade
!pip install chronological
!pip3 install python-dotenv
!pip3 install transformers

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.

In [2]:
pip show openai

Name: openai
Version: 0.25.0
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: /opt/homebrew/lib/python3.10/site-packages
Requires: numpy, openpyxl, pandas, pandas-stubs, requests, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import openai
import requests
from bs4 import BeautifulSoup
import urllib
from bs4.element import Comment
from dotenv import load_dotenv
load_dotenv()
from transformers import GPT2TokenizerFast
openai_api_key = os.environ['OPENAI_API_KEY']
openai.api_key = openai_api_key

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
#Two functions to keep only the visible elements on a webpage (possibly the article written in this page)

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def split_str_halves(string):
  n = len(string)
  print(n)
  global part1
  global part2
  if n%2 == 0:
    part1 = string[0:n//2]
    part2 = string[n//2:]
    return part1,part2
  else:
    part1 = string[0:(n//2+1)]
    part2 = string[(n//2+1):]
    return part1,part2



In [5]:
##STEP 1
#Get the source code of a webpage using BeautifulSoup
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"}
r1 = requests.get('https://www.saagie.com/fr/ingenieur-data-science/', headers=headers)
#print(r1.text)


In [6]:
##STEP 2 
#Keep only the visible elements of the webpage

#print(text_from_html(r1.text))
text_from_webpage = text_from_html(r1.text)


In [ ]:
str1 = "part1"
myVars = vars

In [24]:
##STEP 3
#Split the webpage to stay under GPT3 limitation of 4097 (prompt + result)
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
token_size = False
parts = 0
while token_size == False:    
    if len(tokenizer(text_from_webpage)['input_ids']) > 4097//2 :
    



Token indices sequence length is longer than the specified maximum sequence length for this model (3804 > 1024). Running this sequence through the model will result in indexing errors


NameError: name 'text_from' is not defined

In [8]:
string = "Hello la mif"



In [9]:
split_str_halves('Hello la mif')
print(part1)

12
Hello 


In [23]:
##STEP 4
#Give ChatGPT the webpage with only the visible elements and tell him to keep sentences with subject, verb and complement only


response = openai.Completion.create(
    #prompt='Isole les phrases qui possèdent un sujet, un verbe et un complément dans ce texte' + text_from_webpage,
    prompt=text_from_webpage,
    model="text-davinci-003",
    max_tokens=3000,
    temperature=0.5,
)

# Get the generated text from the response
text = response["choices"][0]["text"]


print(text)


InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 5783 tokens (2783 in your prompt; 3000 for the completion). Please reduce your prompt; or completion length.

In [82]:
#print(text_from_webpage)
text = response["choices"][0]["text"]

print(text)

 Ces dernières années, les entreprises ont compris l’importance d’analyser les données pour en tirer des bénéfices en matière de choix stratégiques, d’accompagnement client et de positionnement vis-à-vis de la concurrence. Pour cela, elles ont créé un pôle data qui regroupe notamment des ingénieurs data science, ou data scientists, pour travailler sur le sujet.

1. Qui est l’ingénieur data science ?

L’ingénieur data science, ou le métier de data scientist, est un nouveau profil apparu cette dernière décennie afin de faire face aux besoins des entreprises en matière de traitement des données.

2. Le rôle de l’ingénieur data science dans votre entreprise

L’ingénieur data science, ou data scientist, a pour but de collecter, de trier, d’organiser, d’analyser et d’exploiter des données afin de résoudre les problématiques l’entreprise comme l’amélioration de la qualité ou de la productivité. Il participe à un projet data avec ses collègues de l’équipe data.

A. Le rôle de l’ingénieur data 

In [83]:
os.remove("output_chat.txt")
file_obj = open("output_chat.txt", "w")
file_obj.write(text)
file_obj.close()